In [16]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import pandas as pd
import datetime

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.metrics import accuracy_score
from scipy.stats import uniform

# these two files are large, and it may take a while to read...
counts = pd.read_csv('data/FremontBridge.csv', index_col='Date', parse_dates=True)
weather = pd.read_csv('data/BicycleWeather.csv', index_col='DATE', parse_dates=True)
daily = counts.resample('d').sum()
daily['Total'] = daily.sum(axis=1)
daily = daily[['Total']] # remove other columns

# add an indicator about Mon - Sun
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
for i in range(7):
    daily[days[i]] = (daily.index.dayofweek == i).astype(float)

# add an indicator about holiday
from pandas.tseries.holiday import USFederalHolidayCalendar
cal = USFederalHolidayCalendar()
holidays = cal.holidays('2012', '2016')
daily = daily.join(pd.Series(1, index=holidays, name='holiday'))

# replace missing data with 0
daily['holiday'].fillna(0, inplace=True)

# This function seems crazy. The main goal is to calculate hours of daylight
# https://www.esrl.noaa.gov/gmd/grad/solcalc/sunrise.html here is an example...
def hours_of_daylight(date, axis=23.44, latitude=47.61):
    """Compute the hours of daylight for the given date"""
    days = (date - pd.datetime(2000, 12, 21)).days
    m = (1. - np.tan(np.radians(latitude))
         * np.tan(np.radians(axis) * np.cos(days * 2 * np.pi / 365.25)))
    return 24. * np.degrees(np.arccos(1 - np.clip(m, 0, 2))) / 180.

daily['daylight_hrs'] = list(map(hours_of_daylight, daily.index))

# temperatures are in 1/10 deg C; convert to C
weather['TMIN'] /= 10
weather['TMAX'] /= 10
weather['Temp (C)'] = 0.5 * (weather['TMIN'] + weather['TMAX'])

# precip is in 1/10 mm; convert to inches
weather['PRCP'] /= 254
weather['dry day'] = (weather['PRCP'] == 0).astype(int)

daily = daily.join(weather[['PRCP', 'Temp (C)', 'dry day']])
# how='left' means calling frame’s index 
# daily = daily.join(weather[['PRCP', 'Temp (C)', 'dry day']], how='left', lsuffix='_left', rsuffix='_right')

# number of years passed
daily['annual'] = (daily.index - daily.index[0]).days / 365.

# Drop any rows with null values
daily.dropna(axis=0, how='any', inplace=True)

column_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun', 'holiday',
                'daylight_hrs', 'PRCP', 'dry day', 'Temp (C)', 'annual']

# Format data into X and Y
X = daily[column_names]
y = daily['Total']

/var/folders/06/68dsc_yn1fx6jzh5kj4hwr7h0000gn/T/ipykernel_78026/4069692651.py:39: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  days = (date - pd.datetime(2000, 12, 21)).days


In [20]:
results = {}

param_grid = {'alpha': uniform()}
rsearch = RandomizedSearchCV(estimator=Ridge(), param_distributions=param_grid, cv=10)
rsearch.fit(X, y)
rbest_alpha = rsearch.best_params_['alpha']
rmean_score = rsearch.best_score_
print('Ridge --- Alpha = {:.9f}, Score = {:.9f}'.format(rbest_alpha, rmean_score))
results['Ridge'] = rmean_score

lsearch = RandomizedSearchCV(estimator=Lasso(tol = .01), param_distributions=param_grid, cv=10)
lsearch.fit(X, y)
lbest_alpha = lsearch.best_params_['alpha']
lmean_score = lsearch.best_score_
print('Lasso --- Alpha = {:.9f}, Score = {:.9f}'.format(lbest_alpha, lmean_score))
results['Lasso'] = lmean_score

model = LinearRegression(fit_intercept=False)
model.fit(X, y)

linear_cv = cross_val_score(model, X, y, cv=10)
lrmean_score = linear_cv.mean()
print('Linear Score = {:.9f}'.format(lrmean_score))
results['Linear'] = lrmean_score

max_score = max(results.values())
print('Best model: {}'.format([k for k, v in results.items() if v == max_score][0]))

Ridge --- Alpha = 0.844471918, Score = 0.769869250
Lasso --- Alpha = 0.895275570, Score = 0.769731840
Linear Score = 0.769175292
Best model: Ridge
